# IT Academy - Data Science amb Python
## Tasca 7: Estructura de control

###  [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)

###  Exercici 1
- Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [1]:
import numpy as np
import pandas as pd
from io import StringIO
import requests
import re

path = '../Data/Web_access_log-akumenius.com.txt'

Logs = pd.read_csv(path, sep='\s | - | \"', names =['DNS','IP','Location','Date','Time','Request','Status','Size','Referer','UserAgent'], skiprows = 261725, engine='python')

Logs

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/f...,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,NaN,NaN,NaN,NaN,NaN
1,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/t...,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,NaN,NaN,NaN,NaN,NaN
2,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,NaN,NaN,NaN,NaN,NaN
3,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,NaN,NaN,NaN,NaN,NaN
4,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
143,www.akumenius.com 5.255.253.53,- [02/Mar/2014:03:05:39 +0100],"GET / HTTP/1.1"" 200 7528","-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...,NaN,NaN,NaN,NaN,NaN
144,www.akumenius.com 74.86.158.107,- [02/Mar/2014:03:09:52 +0100],"HEAD / HTTP/1.1"" 200","""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...,NaN,NaN,NaN,NaN,NaN
145,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",NaN,NaN,NaN,NaN,NaN
146,localhost 127.0.0.1,- [02/Mar/2014:03:10:18 +0100],"OPTIONS * HTTP/1.0"" 200","""-""","Apache (internal dummy connection)"" VLOG=-",NaN,NaN,NaN,NaN,NaN


###  Exercici 2
- Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [2]:
Logs.shape

(148, 10)

In [3]:
Logs.isnull().sum()

DNS            0
IP             0
Location       0
Date           0
Time           0
Request      148
Status       148
Size         148
Referer      148
UserAgent    148
dtype: int64

In [4]:
Logs.UserAgent = Logs.Time
Logs.Request = Logs.Location
Logs.Referer = Logs.Date
Logs.Date = Logs.IP

Logs.head()

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/f...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/f...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/t...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/t...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com 46.222.83.180,- [02/Mar/2014:02:26:14 +0100],GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...


In [5]:
Logs[['DNS','IP']] = Logs.DNS.str.split('\s', expand = True).get([0,1])

In [6]:
Logs.head()

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/f...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/t...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,- [02/Mar/2014:02:26:14 +0100],Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...


In [7]:
Logs.DNS.unique()

array(['www.akumenius.com', 'localhost', 'akumenius.com', 'akumenius.es'],
      dtype=object)

In [8]:
Logs.Time = Logs.Date.str.extract(':(\d{2}:\d{2}:\d{2}.*)]')

In [9]:
Logs.Date= Logs.Date.str.extract('(\d+/\w+/\d+)')

In [10]:
Logs

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/f...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/t...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,NaN,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
...,...,...,...,...,...,...,...,...,...,...
143,www.akumenius.com,5.255.253.53,"GET / HTTP/1.1"" 200 7528",02/Mar/2014,03:05:39 +0100,"GET / HTTP/1.1"" 200 7528",NaN,NaN,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
144,www.akumenius.com,74.86.158.107,"HEAD / HTTP/1.1"" 200",02/Mar/2014,03:09:52 +0100,"HEAD / HTTP/1.1"" 200",NaN,NaN,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
145,localhost,127.0.0.1,"OPTIONS * HTTP/1.0"" 200",02/Mar/2014,03:10:18 +0100,"OPTIONS * HTTP/1.0"" 200",NaN,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"
146,localhost,127.0.0.1,"OPTIONS * HTTP/1.0"" 200",02/Mar/2014,03:10:18 +0100,"OPTIONS * HTTP/1.0"" 200",NaN,NaN,"""-""","Apache (internal dummy connection)"" VLOG=-"


In [11]:
Logs[['Request','Status']] = Logs.Request.str.split('\"', expand = True).get([0,1])

In [12]:
Logs.head()

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/f...,200 8151,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/t...,200 4593,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 2591,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 742,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 9139,NaN,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...


In [13]:
Logs.Size = Logs.Status.str.extract('(\d+$)')
Logs.head()

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/f...,200 8151,8151,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/t...,200 4593,4593,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 2591,2591,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 742,742,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200 9139,9139,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...


In [14]:
Logs.Status = Logs.Status.str.extract('(\d{3})')
Logs.head()


,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/f...,200,8151,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/t...,200,4593,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,2591,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,742,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,9139,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...


###  Exercici 3
- Geolocalitza les IP's. Aqui tens una pagina de interes:
  - [IP2Locattion](https://blog.ip2location.com/knowledge-base/how-to-add-a-sub-account-in-ip2location/)

In [15]:
import json
from ip2geotools.databases.noncommercial import DbIpCity

In [16]:
Ips = Logs.IP.unique()
Ips

array(['46.222.83.180', '127.0.0.1', '157.55.32.185', '74.86.158.106',
       '66.249.76.216', '79.155.78.117', '176.31.255.177', '194.30.26.30',
       '217.174.248.179', '217.12.28.5', '157.55.34.173', '74.86.158.107',
       '180.76.5.200', '157.56.92.174', '217.12.28.130', '188.135.173.80',
       '180.76.6.37', '180.76.5.78', '83.45.25.198', '220.181.108.104',
       '123.125.71.52', '66.249.75.148', '206.198.5.33', '5.255.253.53'],
      dtype=object)

In [17]:
localhost = "127.0.0.1"

def location(ip):
    
    try:
        if ip == localhost:
            return ('Local', 'Local')

        else:
            response = DbIpCity.get(ip, api_key = 'free').to_json()    
            data = json.loads(response)
            return data['city'], data['country']
    
    except:
        return np.nan

In [18]:
location(Logs.IP[1])

('Barcelona', 'ES')

In [19]:
Location = []

for list in Ips:
    
    Location.append(location(list))
    
Location 

#for index, row in Logs.iterrows(): 
        
   #row.Location = location(Logs.IP.unique())
  

[('Barcelona', 'ES'),
 ('Local', 'Local'),
 ('Newark', 'US'),
 ('Chicago', 'US'),
 ('Mountain View', 'US'),
 ('San Cristóbal de La Laguna', 'ES'),
 ('Roubaix', 'FR'),
 ('Ceuta', 'ES'),
 ('Gloucester', 'GB'),
 ('Seville', 'ES'),
 ('Newark', 'US'),
 ('Chicago', 'US'),
 ('Beijing', 'CN'),
 ('Newark', 'US'),
 ('Seville', 'ES'),
 ('Lecce', 'IT'),
 ('Beijing', 'CN'),
 ('Beijing', 'CN'),
 ('Madrid', 'ES'),
 ('Beijing', 'CN'),
 ('Haidian (Haidian Qu)', 'CN'),
 ('Mountain View', 'US'),
 ('Lincolnshire', 'US'),
 ('Moscow (Tsentralnyy administrativnyy okrug)', 'RU')]

In [20]:
Logs

,DNS,IP,Location,Date,Time,Request,Status,Size,Referer,UserAgent
0,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/f...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/f...,200,8151,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
1,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/t...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/t...,200,4593,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
2,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,2591,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
3,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,742,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
4,www.akumenius.com,46.222.83.180,GET /modules/raton/views/themes/bcoos/images/b...,02/Mar/2014,02:26:14 +0100,GET /modules/raton/views/themes/bcoos/images/b...,200,9139,"http://www.akumenius.com/hotel-list?type=e""",Mozilla/5.0 (Linux; Android 4.3; GT-I9300 Buil...
...,...,...,...,...,...,...,...,...,...,...
143,www.akumenius.com,5.255.253.53,"GET / HTTP/1.1"" 200 7528",02/Mar/2014,03:05:39 +0100,GET / HTTP/1.1,200,7528,"-""",Mozilla/5.0 (compatible; YandexBot/3.0; +http:...
144,www.akumenius.com,74.86.158.107,"HEAD / HTTP/1.1"" 200",02/Mar/2014,03:09:52 +0100,HEAD / HTTP/1.1,200,200,"""-""",Mozilla/5.0+(compatible; UptimeRobot/2.0; http...
145,localhost,127.0.0.1,"OPTIONS * HTTP/1.0"" 200",02/Mar/2014,03:10:18 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"
146,localhost,127.0.0.1,"OPTIONS * HTTP/1.0"" 200",02/Mar/2014,03:10:18 +0100,OPTIONS * HTTP/1.0,200,200,"""-""","Apache (internal dummy connection)"" VLOG=-"


###  Exercici 4
- Mostreu-me la teva creativitat, Sorprèn-me fes un pas més enllà amb el analysis anterior.